# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities_df = pd.read_csv("../output_data/CityData.csv")
cities_df.head()


,City,Country,Latitude,Longitude,Max_Temperature(F),Humidity(%),Windspeed(mph),Cloudiness(%),Date
0,saint anthony,US,45.02,-93.22,48.00,56,4.43,1,1604367836
1,albany,US,42.60,-73.97,36.00,67,3.00,98,1604368120
2,ambilobe,MG,-13.20,49.05,67.32,78,4.56,0,1604368121
3,anadyr,RU,64.75,177.48,19.40,92,6.71,75,1604367843
4,rio grande,BR,-32.03,-52.10,60.01,92,3.00,1,1604368122


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# defining humidity and locations for configuring maps
humidity = cities_df["Humidity(%)"].astype(float)
max_humidity = humidity.max()
locations = cities_df[["Latitude", "Longitude"]]

In [15]:
# configuring gmaps with locations and humidity
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=max_humidity, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# Narrow down the cities to fit weather conditions
ideal_city_df = cities_df.loc[(cities_df["Max_Temperature(F)"] > 30) & (cities_df["Max_Temperature(F)"] < 90) 
                              & (cities_df["Windspeed(mph)"] < 10) &(cities_df["Cloudiness(%)"] == 40), :]
# Drop any rows will null values.
ideal_city_df = ideal_city_df.dropna(how='any')
ideal_city_df.reset_index(inplace=True)
del ideal_city_df['index']
ideal_city_df.head()

,City,Country,Latitude,Longitude,Max_Temperature(F),Humidity(%),Windspeed(mph),Cloudiness(%),Date
0,alexandria,EG,31.22,29.96,71.6,56,9.17,40,1604368127
1,camacha,PT,33.08,-16.33,69.8,77,8.05,40,1604368131
2,palmer,US,42.16,-72.33,33.8,50,6.93,40,1604368161
3,touros,BR,-5.20,-35.46,78.8,83,4.70,40,1604368202
4,shimoda,JP,34.67,138.95,64.4,93,8.05,40,1604368257


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_list = []

# use iterrows to iterate through ideal_city dataframe
for index, row in ideal_city_df.iterrows():
    lat = ideal_city_df.loc[index]['Latitude']
    lng = ideal_city_df.loc[index]['Longitude']

# Set parameters to search for hotels with 5000 meters.
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    # Hit the Google Places API for each city's coordinates.    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params)
    
    # extract results
    results = response.json()
    
    # Store the first Hotel result into the DataFrame.
    try:
        hotel_list.append(results['results'][0]['name'])
    except:
        hotel_list.append("")
        #print("------------")

# Add a "Hotel Name" column to the DataFrame.
ideal_city_df["Hotel Name"] = hotel_list
ideal_city_df = ideal_city_df.dropna(how='any')
ideal_city_df
    

,City,Country,Latitude,Longitude,Max_Temperature(F),Humidity(%),Windspeed(mph),Cloudiness(%),Date,Hotel Name
0,alexandria,EG,31.22,29.96,71.6,56,9.17,40,1604368127,Alexandria
1,camacha,PT,33.08,-16.33,69.8,77,8.05,40,1604368131,Vila Baleira
2,palmer,US,42.16,-72.33,33.8,50,6.93,40,1604368161,Palmer
3,touros,BR,-5.20,-35.46,78.8,83,4.70,40,1604368202,Touros
4,shimoda,JP,34.67,138.95,64.4,93,8.05,40,1604368257,Shimoda
5,loiza,PR,18.43,-65.88,82.4,69,4.70,40,1604368335,Canóvanas
6,erzin,TR,36.96,36.20,59.0,100,4.70,40,1604368337,Erzin
7,porto santo,PT,33.07,-16.34,69.8,77,8.05,40,1604368358,Vila Baleira
8,guaruja,BR,-23.99,-46.26,57.2,71,9.17,40,1604368374,Santos
9,marang,MY,5.21,103.21,78.8,94,2.24,40,1604368384,Kuala Terengganu


In [44]:
# # Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)
# fig

In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_city_df.iterrows()]
locations = ideal_city_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))